In [64]:
from lxml import etree
import pandas as pd
from collections import Counter
import os
import glob
import re
import matplotlib.pyplot as plt
import numpy as np
from collections import Counter
from numpy import array
import numpy as np


wdir = "/home/jose/Dropbox/biblia/tb/"
file = "TEIBible" # "*.xml"
outdir = "/home/jose/Dropbox/biblia/tb/resulting data/"

columns = ["len_chapters", "len_verses","len_chars","am_refs","am_1lev_quotes"]
#,"am_diff_people","am_diff_groups","am_diff_places",


In [65]:
parser = etree.XMLParser(encoding='utf-8')

In [66]:
documento_xml = etree.parse(wdir+file+".xml", parser)


In [67]:
documento_root = documento_xml.getroot()

In [68]:
namespaces_concretos = {'tei':'http://www.tei-c.org/ns/1.0','xi':'http://www.w3.org/2001/XInclude'}


In [69]:
books_names = [title for book in documento_root.xpath('//tei:TEI', namespaces=namespaces_concretos, with_tail=True) for title in book.xpath('.//tei:title[1]/text()', namespaces=namespaces_concretos, with_tail=True) ]

In [81]:
for book in documento_root.xpath('//tei:TEI', namespaces=namespaces_concretos, with_tail=True):
        
    print("\n")
    print("title:", book.xpath('.//tei:title[1]/text()', namespaces=namespaces_concretos, with_tail=True))
    print("viaf:", book.xpath('.//tei:title/tei:idno[@type="viaf"]/text()', namespaces=namespaces_concretos, with_tail=True))
    print("code:", book.xpath('.//tei:title/tei:idno[@type="string"]/text()', namespaces=namespaces_concretos, with_tail=True))
    print("verses:", len(book.xpath('.//tei:ab[@type="verse"]', namespaces=namespaces_concretos, with_tail=True)))
    print("chapters:", len(book.xpath('.//tei:div[@type="chapter"]', namespaces=namespaces_concretos, with_tail=True)))
    print("pericopes:", len(book.xpath('.//tei:div[@type="pericope"]', namespaces=namespaces_concretos, with_tail=True)))
    verses = book.xpath('.//tei:ab[@type="verse"]', namespaces=namespaces_concretos, with_tail=True)
    #for verse in verses:
    print(len(verses))
    q_in_verses = []
    rss_in_verses = []
    for verse in  verses:
        q_in_verses.append(len(verse.xpath('.//tei:q', namespaces=namespaces_concretos, with_tail=True)))
        rss_in_verses.append(len(verse.xpath('.//tei:rs', namespaces=namespaces_concretos, with_tail=True)))
    #print(q_in_verses)
    q_in_verses = array(q_in_verses)
    rss_in_verses = array(rss_in_verses)
    #print("q in verses", sorted(q_in_verses))
    print("amount of q", len(q_in_verses))
    print("mean of q in verses", q_in_verses.mean())
    print("median of q in verses", np.median(q_in_verses))
    print("std of q in verses", np.std(q_in_verses, ddof=0))
    print("100th percentile", np.percentile(q_in_verses, q=100))
    position = [i for i,x in enumerate(q_in_verses) if x == 4]
    print(position)
    #print(verses[701].xpath('.//text()', namespaces=namespaces_concretos, with_tail=True))



    grouped_referenced_entities = book.xpath('.//tei:rs/@key', namespaces=namespaces_concretos, with_tail=True)
    print("references:", len(grouped_referenced_entities))
    entities_referenced = [entity for group_entities in grouped_referenced_entities for entity in group_entities.split(" ") ]
    print(len(grouped_referenced_entities))
    print(len(entities_referenced))
    print(len(set(entities_referenced)))
    print(Counter(entities_referenced).most_common(3))
    people_referenced = [entity for entity in entities_referenced if "per" in entity]
    groups_referenced = [entity for entity in entities_referenced if "org" in entity]
    places_referenced = [entity for entity in entities_referenced if "pla" in entity]
    times_referenced = [entity for entity in entities_referenced if "tim" in entity]
    works_referenced = [entity for entity in entities_referenced if "wor" in entity]

    print(Counter(people_referenced).most_common(1))
    print(len(people_referenced))
    print(len(set(people_referenced)))

    print(Counter(groups_referenced).most_common(1))
    print(len(groups_referenced))
    print(len(set(groups_referenced)))

    print(Counter(places_referenced).most_common(1))
    print(len(places_referenced))
    print(len(set(places_referenced)))

    print(Counter(times_referenced).most_common(1))
    print(len(times_referenced))
    print(len(set(times_referenced)))

    print(Counter(works_referenced).most_common(1))
    print(len(works_referenced))
    print(len(set(works_referenced)))

    print("quotations:", len(book.xpath('.//tei:q', namespaces=namespaces_concretos, with_tail=True)))
    print("1 level quotations:", len(book.xpath('.//tei:ab[@type="verse"]/tei:q', namespaces=namespaces_concretos, with_tail=True)))

    
    grouped_who = book.xpath('.//tei:q/@who', namespaces=namespaces_concretos, with_tail=True)
    who_entities = [entity for group_who in grouped_who for entity in group_who.split(" ") ]
    print(len(who_entities))
    print(len(set(who_entities)))
    
    grouped_toWohm = book.xpath('.//tei:q/@corresp', namespaces=namespaces_concretos, with_tail=True)
    toWhom_entities = [entity for group_toWohm in grouped_toWohm for entity in group_toWohm.split(" ") ]
    print(len(toWhom_entities))
    print(len(set(toWhom_entities)))

    q_oral = book.xpath('.//tei:q[@type="oral"]', namespaces=namespaces_concretos, with_tail=True)
    q_dream = book.xpath('.//tei:q[@type="dream"]', namespaces=namespaces_concretos, with_tail=True)
    q_prayer = book.xpath('.//tei:q[@type="prayer"]', namespaces=namespaces_concretos, with_tail=True)
    q_oath = book.xpath('.//tei:q[@type="oath"]', namespaces=namespaces_concretos, with_tail=True)
    q_written = book.xpath('.//tei:q[@type="written"]', namespaces=namespaces_concretos, with_tail=True)
    q_song = book.xpath('.//tei:q[@type="song"]', namespaces=namespaces_concretos, with_tail=True)
    print(len(q_oral))
    print(len(q_dream))
    print(len(q_prayer))
    print(len(q_oath))
    print(len(q_written))
    print(len(q_song))
    




title: ['Génesis']
viaf: ['174582712']
code: ['GEN']
verses: 1533
chapters: 50
pericopes: 79
1533
amount of q 1533
mean of q in verses 0.6131767775603392
median of q in verses 1.0
std of q in verses 0.6974063538354425
100th percentile 4.0
[701, 1144]
references: 4992
4992
5285
497
[('#per14', 419), ('#per4', 394), ('#per2', 257)]
[('#per14', 419)]
3956
349
[('#org88', 198)]
529
59
[('#pla5', 139)]
793
86
[('#tim3', 4)]
7
3
[]
0
0
quotations: 940
1 level quotations: 840
948
57
1078
68
856
11
15
0
1
0


title: ['Éxodo']
viaf: ['265859281']
code: ['EXO']
verses: 1213
chapters: 40
pericopes: 87
1213
amount of q 1213
mean of q in verses 0.877164056059357
median of q in verses 1.0
std of q in verses 0.7856385309813977
100th percentile 3.0
[]
references: 3035
3035
3057
165
[('#per14', 553), ('#org70', 543), ('#per26', 320)]
[('#per14', 553)]
1429
80
[('#org70', 543)]
1021
39
[('#pla5', 201)]
530
34
[('#tim9', 24)]
52
9
[('#wor7', 21)]
25
3
quotations: 1064
1 level quotations: 800
1110
21
11

In [58]:
books_names[0:3]

['Lucas']

In [59]:
metadata_bible = pd.DataFrame(index = books_names, columns = columns).fillna(0)

In [60]:
metadata_bible

,len_chapters,len_verses,len_chars,am_refs,am_1lev_quotes
Lucas,0,0,0,0,0
